In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.cluster import KMeans

## Puntaje
Your submission scored 0.87339, which is not an improvement of your best score. Keep trying!

In [2]:
df1 = pd.read_csv('../data/features_basicas.csv', low_memory=False, index_col='person')
df2 = pd.read_csv('../data/features_checkouts.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('../data/features_vistos.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('../data/feature_dispositivo_mas_usado.csv', low_memory=False, index_col='person')
df5 = pd.read_csv('../data/feature_dia_mas_activo.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('../data/feature_compra_onsite.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('../data/feature_cantidad_de_eventos_en_intervalos.csv', low_memory=False, index_col='person')
df8 = pd.read_csv('../data/feature_dispositivos_storage.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('../data/featureUsuarioSonDeSaoPablo.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('../data/featureCantidadMaximaPersonaVeUnProducto.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('../data/features_basicas_ult_quin.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('../data/features_ultima_ocurrencia_cada_evento.csv', low_memory=False, index_col='person')
df13 = pd.read_csv('../data/days_elapsed_from_last_event.csv', low_memory=False, index_col='person')
df14 = pd.read_csv('../data/featureUsuarioRealiza30EventosEn20MinsLosUltimos2Dias.csv', low_memory=False, index_col='person')
df15 = pd.read_csv('../data/features_ultimo_checkout.csv', low_memory=False, index_col='person')
df16 = pd.read_csv('../data/diferencia_de_count_de_eventos.csv', low_memory=False, index_col='person')
df17 = pd.read_csv('../data/top_10_celulares.csv', low_memory=False, index_col='person')
df18 = pd.read_csv('../data/feature_cantidad_de_eventos_en_intervalos_ult_quin.csv', low_memory=False, index_col='person')

labels = pd.read_csv('../data/labels_training_set.csv', low_memory=False, index_col='person')

df_unidos = df1.join(df2, how='inner')
df_unidos = df_unidos.join(df3, how='inner')
df_unidos = df_unidos.join(df4, how='inner')
df_unidos = df_unidos.join(df5, how='inner')
df_unidos = df_unidos.join(df6, how='inner')
df_unidos = df_unidos.join(df7, how='inner')
df_unidos = df_unidos.join(df8, how='inner')
df_unidos = df_unidos.join(df9, how='inner')
df_unidos = df_unidos.join(df10, how='inner')
df_unidos = df_unidos.join(df11, how='inner')
df_unidos = df_unidos.join(df12, how='inner')
df_unidos = df_unidos.join(df13, how='inner')
df_unidos = df_unidos.join(df14, how='inner')
df_unidos = df_unidos.join(df15, how='inner')
# Al que tenga "diferencia_de_count_de_eventos.csv" hay que rellenarlo
df_unidos = df_unidos.join(df16).fillna(0)
df_unidos = df_unidos.join(df17, how='inner')
df_unidos = df_unidos.join(df18, how='inner')

In [3]:
features = ['cant_viewed_product'
,'days_elapsed'
,'storage_mas_checkout'
,'ult_conversion'
,'ult_checkout'
,'ult_viewed product'
,'tasa_ult_quin_cant_checkouts'
,'dispositivo_Computer'
,'cant_eventos_21_a_3hs'
,'tasa_ult_quin_cant_entradas_ads'
,'ult_search engine hit'
,'cant_eventos_11_a_14hs'
,'ult_searched products'
,'tasa_ult_quin_cant_searched_product'
,'cant_brand_listings'
,'ult_brand listing'
,'model_encoding'
,'ult_quin_cant_eventos_21_a_3hs'
,'total_sesiones'
,'cant_checkouts'
,'ult_generic listing'
,'storage_mas_eventos'
,'tasa_ult_quin_cant_visitas'
,'ult_quin_cant_eventos_4_a_10hs'
,'vio_mas_Apple'
,'tasa_ult_quin_cant_brand_listings'
,'diff_viewed_products_15_dias'
,'tiene_conversions'
,'cant_conversions'
,'cant_generic_listings'
,'cant_eventos_15_a_20hs'
,'cant_entradas_ads'
,'tasa_ult_quin_cant_entradas_buscador'
,'cant_searched_product'
,'dispositivo_Smartphone'
,'ult_ad campaign hit'
,'ult_quin_cant_viewed_product'
,'ult_quin_cant_eventos_11_a_14hs'
,'promedio_eventos_por_sesion'
,'tasa_ult_quin_cant_viewed_product']

## Kmeans magic

In [4]:
puntos = df_unidos[features]
puntos_norm = (puntos - puntos.mean()) / puntos.std()

kmeans = KMeans(n_clusters=2, n_jobs=-1)
df_unidos['clases_kmeans'] = kmeans.fit_predict(puntos_norm)
features.append('clases_kmeans')
len(features)

41

In [5]:
#  Le asigno las features al subset de usuarios que nos da Trocafone para entrenar.
df_test = df_unidos.join(labels, on='person', how='inner')
df_test.shape

(19414, 101)

In [6]:
X = df_test[features]
Y = df_test['label']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=311)

In [21]:
param_grid = { 
    'n_estimators': [x for x in range(100, 500, 100)],
    'gamma' : [5,6,7,8,9,10],
    'max_depth': [3,4,5]
}

In [22]:
xgb_kmeans = XGBClassifier(random_state=80)
CV_xgb_kmeans = GridSearchCV(estimator=xgb_kmeans, param_grid=param_grid, cv=5, scoring='roc_auc', n_jobs=4,)
CV_xgb_kmeans.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=80, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'n_estimators': [100, 200, 300, 400], 'gamma': [5, 6, 7, 8, 9, 10], 'max_depth': [3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [23]:
CV_xgb_kmeans.best_params_

{'gamma': 6, 'max_depth': 4, 'n_estimators': 200}

In [24]:
CV_xgb_kmeans.best_score_

0.8762794383397318

In [25]:
print(classification_report(y_test, CV_xgb_kmeans.predict(X_test)))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      3675
           1       0.38      0.02      0.05       208

   micro avg       0.95      0.95      0.95      3883
   macro avg       0.67      0.51      0.51      3883
weighted avg       0.92      0.95      0.92      3883



In [26]:
df_submit = pd.read_csv('../data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos[features], how='inner')

In [27]:
df_events.shape

(19415, 41)

In [28]:
# Notar que la predicción está hecha con el modelo pasado por GridSearchCV y no el otro que está sin kfoldear.
kaggle_pred = CV_xgb_kmeans.predict_proba(df_events)
proba_de_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = series.values

In [29]:
df_submit.to_csv('../submit_xgb_v7_kmeans_ndv.csv')